In [ ]:
# This is an example of node classification by multilayer graphical network via pytorch 

import argparse
from utils import load_data
from model import mGCN
import torch.nn as nn
import torch.optim as optim
import torch
from dataset import Dataset
from sklearn.metrics import roc_auc_score
import numpy as np
import random
import copy
import torch.nn.functional as F
from sklearn.metrics import accuracy_score, roc_auc_score, recall_score, f1_score

parser = argparse.ArgumentParser(description='GCN')

parser.add_argument('--dataset', type=str, default='acm')
parser.add_argument('--fast_split', action='store_true',
                    help="for large custom datasets (not OGB), do a fast data split")

parser.add_argument('--runs', type=int, default=1)
parser.add_argument('--hidden', type=int, default=128,
                    help='Number of hidden units.')
parser.add_argument('--epochs', type=int, default=200,
                    help='Number of training epochs.')
parser.add_argument('--alpha', type=float, default=0.6,
                    help='Hyperparameter')
parser.add_argument('--dropout', type=float, default=0.5,
                    help='Dropout')
parser.add_argument('--training_ratio', type=float, default=0.3,
                    help='Training Ratio')
parser.add_argument('--lr', type=float, default=0.001,
                    help='Learning Rate')
parser.add_argument('--weight_decay', type=float, default=1e-2,
                    help='Weight_decay')
parser.add_argument('--test_view', type=int, default=1,
                    help='Number of training epochs.')
parser.add_argument("-f", "--fff", help="a dummy argument to fool ipython", default="1")

args = parser.parse_args()


def get_accuracy(output, labels):
    """
    Accuracy calculation method
    """
    preds = output.max(1)[1].type_as(labels)
    correct = preds.eq(labels).double()
    correct = correct.sum()
    return correct / len(labels)

def train():
    model_mGCN.train()
    optimizer.zero_grad()
    out = model_mGCN(sample_data.dataset)
    loss_train = F.cross_entropy(out[sample_data.training_id], sample_data.labels[sample_data.training_id])
    acc_train = get_accuracy(out[sample_data.training_id], sample_data.labels[sample_data.training_id])
    
    loss_train.backward()
    optimizer.step()
    

    
    loss_val = F.cross_entropy(out[sample_data.valid_id], sample_data.labels[sample_data.valid_id])
    acc_val = get_accuracy(out[sample_data.valid_id], sample_data.labels[sample_data.valid_id])
    
    print('Epoch: {:04d}'.format(epoch+1),
          'loss_train: {:.4f}'.format(loss_train.item()),
          'acc_train: {:.4f}'.format(acc_train.item()),
          'loss_val: {:.4f}'.format(loss_val.item()),
          'acc_val: {:.4f}'.format(acc_val.item()))

@torch.no_grad()
def test():
    model_mGCN.eval()
    out = model_mGCN(sample_data.data)

    loss_test = F.nll_loss(out[sample_data.test_id], sample_data.labels[sample_data.test_id])
    acc_test = get_accuracy(out[sample_data.test_id], sample_data.labels[sample_data.test_id])
    
    print("Test set results:",
          "loss= {:.4f}".format(loss_test.item()),
          "accuracy= {:.4f}".format(acc_test.item()))
    
sample_data = Dataset(args.dataset,train_percent=args.training_ratio,valid_percent = 0.1)

#data, num_views, training_id, valid_id, test_id, num_classes, labels, adj_list, edge_list = load_data(args.dataset, training_percent=args.training_ratio)
# data, split_edges = split_data(data_ori, args.test_view, multi=True)
print("Finish loading data")
num_feat = sample_data.dataset.x.shape[1]
model_mGCN = mGCN(num_feat, args.hidden, None, sample_data.num_dims, args.alpha, sample_data.num_classes, dropout=args.dropout)
criterion = nn.NLLLoss()
optimizer = optim.Adam(model_mGCN.parameters(), lr=args.lr)

    
for epoch in range(args.epochs):
    train()
    
print("Model training is complete")

test()

Finish loading data
Epoch: 0001 loss_train: 1.1524 acc_train: 0.3230 loss_val: 1.1339 acc_val: 0.3576
Epoch: 0002 loss_train: 1.1266 acc_train: 0.3230 loss_val: 1.1497 acc_val: 0.2848
Epoch: 0003 loss_train: 1.1407 acc_train: 0.3506 loss_val: 1.1540 acc_val: 0.3576
Epoch: 0004 loss_train: 1.1134 acc_train: 0.3451 loss_val: 1.1233 acc_val: 0.3675
Epoch: 0005 loss_train: 1.1044 acc_train: 0.3451 loss_val: 1.1130 acc_val: 0.3245
Epoch: 0006 loss_train: 1.1013 acc_train: 0.3297 loss_val: 1.1053 acc_val: 0.2815
Epoch: 0007 loss_train: 1.1000 acc_train: 0.3264 loss_val: 1.1019 acc_val: 0.2914
Epoch: 0008 loss_train: 1.0995 acc_train: 0.3241 loss_val: 1.0976 acc_val: 0.3841
Epoch: 0009 loss_train: 1.0992 acc_train: 0.3407 loss_val: 1.0964 acc_val: 0.3377
Epoch: 0010 loss_train: 1.1009 acc_train: 0.3208 loss_val: 1.0983 acc_val: 0.3477
Epoch: 0011 loss_train: 1.1003 acc_train: 0.3197 loss_val: 1.0964 acc_val: 0.3543
Epoch: 0012 loss_train: 1.0991 acc_train: 0.3197 loss_val: 1.0980 acc_val: 0.3

Epoch: 0101 loss_train: 0.8251 acc_train: 0.6759 loss_val: 0.8368 acc_val: 0.6490
Epoch: 0102 loss_train: 0.7706 acc_train: 0.7420 loss_val: 0.8011 acc_val: 0.6921
Epoch: 0103 loss_train: 0.7738 acc_train: 0.7321 loss_val: 0.8219 acc_val: 0.6821
Epoch: 0104 loss_train: 0.7222 acc_train: 0.7541 loss_val: 0.7540 acc_val: 0.7185
Epoch: 0105 loss_train: 0.6837 acc_train: 0.8037 loss_val: 0.7197 acc_val: 0.7285
Epoch: 0106 loss_train: 0.6823 acc_train: 0.8026 loss_val: 0.7140 acc_val: 0.8146
Epoch: 0107 loss_train: 0.6357 acc_train: 0.7718 loss_val: 0.6753 acc_val: 0.7252
Epoch: 0108 loss_train: 0.5973 acc_train: 0.7949 loss_val: 0.6287 acc_val: 0.7616
Epoch: 0109 loss_train: 0.5922 acc_train: 0.8534 loss_val: 0.6299 acc_val: 0.8311
Epoch: 0110 loss_train: 0.5504 acc_train: 0.8203 loss_val: 0.5843 acc_val: 0.7781
Epoch: 0111 loss_train: 0.5229 acc_train: 0.8379 loss_val: 0.5651 acc_val: 0.7881
Epoch: 0112 loss_train: 0.5085 acc_train: 0.8875 loss_val: 0.5435 acc_val: 0.8477
Epoch: 0113 loss

In [9]:
import dataset

In [10]:
dataset

<module 'dataset' from '/Users/hanzhengwang/Desktop/mGCN_Toolbox/dataset.py'>

In [11]:
dataset.dataset

AttributeError: module 'dataset' has no attribute 'dataset'